# Notebook 4: Upload a Dataset

## Setup Variables

Before we start let's store some variables that will come in handy later in the notebook.

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
# autodetect the host_ip
from utils import auto_detect_domain_host_ip

DOMAIN_HOST_IP = auto_detect_domain_host_ip()

In [ ]:
# Set the email and password of your Domain node.
# We will be using the default email and password that got created during Domain creation.

ADMIN_EMAIL="info@openmined.org"
ADMIN_PASSWORD="changethis"

## Step 1: Log into our Domain

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
import syft as sy

print(f"You're running syft version: {sy.__version__}")

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
# Let's log into the domain using the credentials
try:
    domain_client = sy.login(
        url=DOMAIN_HOST_IP, email=ADMIN_EMAIL, password=ADMIN_PASSWORD
    )
    print()
    print("🎉 You successfully connected to your domain!")
except Exception as e:
    print("❌ Unable to connect, did you set the `DOMAIN_HOST_IP` variable above?")
    raise e

## Step 2: Creating a Dataset

<img src="img/tab_info.png" alt="tab" style="width: 100px; margin:0;" />

### MedNIST Dataset

We will be using the MedNIST dataset. The MedNIST dataset was gathered from several sets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset.

The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license. If you use the MedNIST dataset, please acknowledge the source, e.g. https://colab.research.google.com/drive/1wy8XUSnNWlhDNazFdvGBHLfdkGvOHBKe#scrollTo=ZaHFhidyCBJa

Let's move on to download and extract the dataset.

The dataset has been stored pickle file. Lets download the dataset using the instructions below.

Copy your variables `MY_DATASET_URL` from your session details.

```
Hi Person,
These are your Session Details:
-------------------------------
Username: azureuser
Password: **********
VM IP Address: x.x.x.x
    
📎 MY_DATASET_URL:
'https://media.githubusercontent.com/media/shubham3121/datasets/main/MedNIST/subsets/xxxxxxx.pkl'

```

<img src="img/tab_edit.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
# replace these with your own from the session details
# paste your dataset url here
MY_DATASET_URL = ""

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
# download MedNIST.pkl
from utils import download_mednist_dataset

FILE_PATH = download_mednist_dataset(MY_DATASET_URL)

Now, before we move forward, let's store some variables related to the dataset.

We require your participant number and the total participant count in the session to allocate you a unique subset of the MedNIST data.

### Load the Dataset

Below are some helper methods, thatwe will require to load the dataset.

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
# Import helper methods
from syft.core.adp.data_subject_list import DataSubjectList
from utils import (
    get_data_description,
    get_label_mapping,
    split_into_train_test_val_sets,
    load_data_as_df,
    get_dataset_name,
)

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
# Let's load the dataset as a dataframe
dataset_df = load_data_as_df(FILE_PATH)

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
# Let's get a peek of the dataset
dataset_df.head()

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
# Split the dataset into `train`, `validation` and `test` sets.
data_dict = split_into_train_test_val_sets(dataset_df)

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
data_dict["train"].shape, data_dict["val"].shape, data_dict["test"].shape

Get the dataset description, that needs to be provided to the domain while uploading the dataset.

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
dataset_description = get_data_description(dataset_df)
print(dataset_description)

We can see that dataset description contains a brief info about the dataset and also a few meta information related to the dataset.

### Prepare Dataset for Upload

In the next step we will create assets for our datasets. Asset is a collection of private data. In our case the images and labels in the train, val and test sets will be part of the assets.

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
import numpy as np

assets = dict()

for name, data in data_dict.items():

    # Let's create data subjects list.
    # Data Subjects are the individuals whose privacy we're trying to protect.
    data_subjects = DataSubjectList.from_series(data["patient_id"])

    # Convert images to numpy int64 array
    images = data["image"]
    images = np.dstack(images.values).astype(np.int64)  # type cast to int64
    dims = images.shape
    images = images.reshape(dims[0] * dims[1], dims[2])  # reshape to 2D array
    images = np.rollaxis(images, -1)

    # Convert labels to numpy int64 array
    labels = data["label"].to_numpy().astype("int64")

    # Next we will make your data private private with min, max and data subjects.
    # The min and max are minimum and maximum value in the given data.

    # converting images to private data
    image_data = sy.Tensor(images).private(
        min_val=0, max_val=255, data_subjects=data_subjects
    )

    # converting labels to private data
    label_data = sy.Tensor(labels).private(
        min_val=0, max_val=5, data_subjects=data_subjects
    )

    assets[f"{name}_images"] = image_data
    assets[f"{name}_labels"] = label_data

Finally, we will upload the assets to the domain.

## Step 3: Upload the Dataset

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
# creating/uploading the dataset
# Name of the dataset
# Gets the dataset name from the dataset url
name = get_dataset_name(MY_DATASET_URL)

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
# upload the MedNIST data
domain_client.load_dataset(
    assets=assets, name=name, description=dataset_description, use_blob_storage=True
)

Now let's check if the dataset we successfully uploaded

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
domain_client.datasets

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
domain_client.datasets[0]

## Step 4: Create a Data Scientist Account

<img src="img/tab_edit.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
data_scientist_details = {
    "name": "Samantha Carter",
    "email": "sam@sg1.net",
    "password": "stargate",
    "budget": 9999,
}

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
domain_client.users.create(**data_scientist_details)

<img src="img/tab_run.png" alt="tab" style="width: 100px; margin:0;" />

In [ ]:
print("Please give these details to the data scientist 👇🏽")
login_details = {}
login_details["url"] = DOMAIN_HOST_IP
login_details["name"] = data_scientist_details["name"]
login_details["email"] = data_scientist_details["email"]
login_details["password"] = data_scientist_details["password"]
login_details["dataset_name"] = name
print()
print(login_details)

<img src="img/tab_copy_send.png" alt="tab" style="width: 123px; margin:0;" />

Please copy the output of the above Run cell and share it with your Data Scientist or Demo Coordinator.

<img src="img/tab_finish.png" alt="tab" style="width: 100px; margin:0;" />

🙌🏽 Notebook Complete!

🖐 Raise your hand in Zoom

👉🏽 Then, click to continue to Notebook 5: [05-complete.ipynb](05-complete.ipynb)